In [15]:
import json
import requests
import os

from folium import Map, TileLayer

In [16]:
cwd = str(os.getcwd())
cog_path = 'file://' + cwd +'/cog/mc_1000_byte_cog.tif'

In [17]:
titiler_endpoint = "http://127.0.0.1:8000"
url = cog_path

# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = requests.get(
    f"{titiler_endpoint}/info",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
print(r)

{'bounds': [-177.95454545454567, -55.77657301866767, 175.90909090909065, 21.943045533438067], 'band_metadata': [['1', {}]], 'band_descriptions': [['1', 'treecover2019']], 'dtype': 'uint8', 'nodata_type': 'Nodata', 'colorinterp': ['gray']}


In [21]:
r = requests.get(
    f"{titiler_endpoint}/tilejson.json",
    params = {
        "url": url,
        "colormap_name": "greens"
    }
).json()


m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=5
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="test"
)
aod_layer.add_to(m)
m

In [4]:
print(m)